# Building a Multiclass Classifier Based on the Fashion MNIST Dataset

In [41]:
#Import the Fashion MNIST dataset from TensorFlow
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image \
import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

Source: [Fashion MNIST dataset](https://knowyourdata-tfds.withgoogle.com/#dataset=fashion_mnist&tab=STATS&select=default_segment.fashion_mnist.label.value&expanded_groups=fashion_mnist)

Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.

In [42]:
#Load the Fashion MNIST dataset
(features_train, label_train), (features_test, label_test) = \
fashion_mnist.load_data()

In [43]:
print('Number of features train: ',len(set(label_train)),'\n'
      'Number of features test: ',len(set(label_test)))

Number of features train:  10 
Number of features test:  10


In [44]:
#List the values that the train label can take
label_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [45]:
#List the values that the test label can take
label_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [46]:
#Print out the shape of features_train and features_test.
print('Features train: ',features_train.shape, '\n'
      'Features test: ', features_test.shape)

Features train:  (60000, 28, 28) 
Features test:  (10000, 28, 28)


In [47]:
#Reshape these to add a new dimension corresponding to the number of colours. For grayscale is 1
features_train = features_train.reshape(60000, 28, 28, 1)
features_test = features_test.reshape(10000, 28, 28, 1)

In [48]:
#Define batch size, image height and image width variables
batch_size = 16
img_height = 28
img_width = 28

In [49]:
"""
Create a data generator with the following data augmentation for training:
rescale=1./255, 
rotation_range=40, 
width_shift_range=0.1, 
height_shift_range=0.1, 
shear_range=0.2, 
zoom_range=0.2, 
horizontal_flip=True, 
fill_mode='nearest'
"""
train_img_gen = ImageDataGenerator(rescale=1./255, \
                                   rotation_range=40, \
                                   width_shift_range=0.1, \
                                   height_shift_range=0.1, \
                                   shear_range=0.2, \
                                   zoom_range=0.2, \
                                   horizontal_flip=True, \
                                   fill_mode='nearest')

In [50]:
#Create another data generator for testing that only has rescaling
val_img_gen = ImageDataGenerator(rescale=1./255)

In [51]:
#Create a data generator called val_data_gen using 
train_data_gen = train_img_gen.flow(features_train, \
                                    label_train, \
                                    batch_size=batch_size)

In [52]:
#Create a data generator called val_data_gen
val_data_gen = train_img_gen.flow(features_test, \
                                  label_test, \
                                  batch_size=batch_size)

In [53]:
#Set a random seed for numpy and tensorflow
np.random.seed(8)
tf.random.set_seed(8)

In [54]:
"""
Create a neural network architecture with the following layers: 
* a convolutional layer with Conv2D(64, (3,3), activation='relu') 
  * followed by MaxPooling2D(2,2);
* a convolutional layer with Conv2D(64, (3,3), activation='relu') 
  * followed by MaxPooling2D(2,2); a flatten layer;
* a fully connected layer with Dense(128, activation='relu');
* a fully connected layer with Dense(10, activation='softmax')
"""
model = tf.keras.Sequential\
        ([layers.Conv2D(64, (3,3), activation='relu',\
                        input_shape=(img_height, \
                                     img_width ,1)), \
                        layers.MaxPooling2D((2,2)), \
                        layers.Conv2D(128, (3,3),\
                                      activation='relu'),\
                        layers.MaxPooling2D((2,2)),\
                        layers.Flatten(), \
                        layers.Dense(128, \
                                     activation='relu'), \
                        layers.Dense(10, \
                                     activation='softmax')])

In [55]:
#Compile the model using an Adam optimizer with a learning rate of 0.001.
optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss='sparse_categorical_crossentropy', \
              optimizer=optimizer, metrics=['accuracy'])

In [56]:
#Train the model and evaluate it on the testing set.
model.fit_generator(train_data_gen, \
                    steps_per_epoch=len(features_train) \
                                    // batch_size, \
                    epochs=5, \
                    validation_data=val_data_gen, \
                    validation_steps=len(features_test) \
                                     // batch_size)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


3750/3750 [==============================] - 187s 50ms/step - loss: 0.7986 - accuracy: 0.7013 - val_loss: 0.6313 - val_accuracy: 0.7640
Epoch 2/5
3750/3750 [==============================] - 186s 50ms/step - loss: 0.5788 - accuracy: 0.7824 - val_loss: 0.5509 - val_accuracy: 0.7953
Epoch 3/5
3750/3750 [==============================] - 185s 49ms/step - loss: 0.5189 - accuracy: 0.8065 - val_loss: 0.5340 - val_accuracy: 0.7945
Epoch 4/5
3750/3750 [==============================] - 184s 49ms/step - loss: 0.4800 - accuracy: 0.8194 - val_loss: 0.4714 - val_accuracy: 0.8228
Epoch 5/5
3750/3750 [==============================] - 186s 50ms/step - loss: 0.4543 - accuracy: 0.8294 - val_loss: 0.4806 - val_accuracy: 0.8267


**Observation:**  

This CNN model was trained on 60,000 samples, of which 10,000 were used for testing. After training for five epochs, the following accuracy scores where achived: 

* The training set accuracy was 0.8294.  
* The validation set accuracy was 0.8267.  